Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [4]:
import numpy as np
from icecream import ic
from copy import deepcopy
from tqdm.auto import tqdm

In [4]:
NUM_KNAPSACKS = 2
NUM_ITEMS = 10
NUM_DIMENSIONS = 2

In [5]:
VALUES = np.random.randint(0, 100, size=NUM_ITEMS)
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = np.random.randint(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)

In [8]:
CONSTRAINTS

array([446, 235], dtype=int32)

In [9]:
# A random solution
solution = np.array(
    [np.random.random(NUM_ITEMS) < 0.5 for _ in range(NUM_KNAPSACKS)], dtype=np.bool
)

In [10]:
solution

array([[ True, False, False,  True,  True, False,  True,  True, False,
         True],
       [False, False,  True, False,  True, False, False,  True, False,
         True]])

In [65]:
# Check that the same object does not appear in multiple knapsacks
np.all(solution.sum(axis=0) <= 1)

np.False_

In [66]:
# Check if the solution is valid
all_knapsacks = np.any(solution, axis=0)
np.all(WEIGHTS[all_knapsacks].sum(axis=0) < CONSTRAINTS)

np.False_

## TEST PROBLEMS

In [140]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)

In [142]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)

In [144]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)

In [ ]:
# V = total value of the items in the knapsacks
def V(solution):
    sum = 0
    for knapsack in range(NUM_KNAPSACKS):
        for item in range(NUM_ITEMS):
            sum += solution[knapsack, item] * VALUES[item]
    return sum


# P = penalty for invalid knapsacks
# P = number of invalid knapsacks * total invalid weight
def P(solution):
    # npk = number of invalid knapsacks
    npk = 0
    # tiw = total invalid weight
    tiw = 0
    for knapsack in range(NUM_KNAPSACKS):
        for dim in range(NUM_DIMENSIONS):
            total_weigth= 0
            for item in range(NUM_ITEMS):
                total_weigth += solution[knapsack, item] * WEIGHTS[item, dim]
            if total_weigth > CONSTRAINTS[knapsack, dim]:
                npk += 1
                tiw += total_weigth - CONSTRAINTS[knapsack, dim]
    return npk*tiw
# D = penalty for duplicate items in different knapsacks
# D = number of duplicated items * values of the items
def D(solution):
    d = 0
    for item in range(NUM_ITEMS):
        count = 0
        for knapsack in range(NUM_KNAPSACKS):
            if solution[knapsack, item]:
                count += 1
        if count > 1:
            d += count * VALUES[item]
    return d

In [106]:
# Optimization of the functions by ChatGPT

def V(solution):
    return float((solution @ VALUES).sum())

def P(solution):
    W = WEIGHTS if WEIGHTS.ndim == 2 else WEIGHTS[:, None]           # (N,D)
    C = CONSTRAINTS if CONSTRAINTS.ndim == 2 else CONSTRAINTS[None]  # (K,D)
    excess = np.clip(solution @ W - C, 0, None)                      # (K,D) solo sfori
    return float(excess.sum())

def D(solution):
    counts = solution.sum(axis=0)             # (N,)
    excess = np.clip(counts - 1, 0, None)     # (N,)
    return float((excess * VALUES).sum())

def _W2d(WEIGHTS):
    return WEIGHTS if WEIGHTS.ndim == 2 else WEIGHTS[:, None]

In [107]:
def cost(solution):
    return (P(solution)+D(solution))-V(solution)

def tweak(solution, strength: float=0.1):
    new_solution = deepcopy(solution)
    again = True
    while again:

        # case 1: remove all duplicates of an object expect for one casual knapsack
        if D(new_solution) > 0:
            counts = new_solution.sum(axis=0)               
            dup_items = np.where(counts > 1)[0]   
            for j in dup_items:
                ks = np.where(new_solution[:, j])[0]        
                keep = rng.choice(ks)         
                new_solution[ks, j] = False
                new_solution[keep, j] = True
        # case 2: remove the worst weigth/value object from a casual knapsack that is invalid
        if P(new_solution) > 0: 
            W = _W2d(WEIGHTS)                      
            K = new_solution.shape[0]
            loads = new_solution @ W                        
            for k in range(K):
                while np.any(loads[k] > CONSTRAINTS):
                    items_in = np.where(new_solution[k])[0]
                    if items_in.size == 0:
                        break 
                    weights_items = W[items_in] 
                    norm_w = weights_items / np.maximum(CONSTRAINTS, 1)
                    score = norm_w.sum(axis=1) / np.maximum(VALUES[items_in], 1e-9)
                    j = items_in[np.argmax(score)]
                    new_solution[k, j] = False
                    loads[k] = new_solution[k] @ W
        # case 3: casualy remove/add an object from a casual knapsack
        if P(new_solution) == 0 and D(new_solution) == 0:
            knapsack = rng.integers(0, NUM_KNAPSACKS-1)
            item = rng.integers(0, NUM_ITEMS-1)
            new_solution[knapsack, item] = not new_solution[knapsack, item]
        again = rng.random() < strength
        
    return new_solution

In [ ]:
# The solution shown is for PROBLEM 3 but the algorithm also works for PROBLEM 1 and PROBLEM 2

current_solution = solution = np.array(
    [np.random.random(NUM_ITEMS) < 0.5 for _ in range(NUM_KNAPSACKS)], dtype=np.bool
)

current_cost = cost(current_solution)
current_d_cost=D(current_solution)
current_p_cost=P(current_solution)

MAX_STEP = 1000
temperature = 100

for step in tqdm(range(MAX_STEP)):
    new_solution = tweak(current_solution, strength=0.3)
    new_cost = cost(new_solution)

    new_d_cost = D(new_solution)
    new_p_cost = P(new_solution)


    if new_cost < current_cost and (new_d_cost <= current_d_cost and new_p_cost <= current_p_cost):
        current_solution = new_solution
        current_cost = new_cost
        current_p_cost = new_p_cost
        current_d_cost = new_d_cost
        ic(step, current_cost)
    elif new_cost == current_cost and (new_d_cost <= current_d_cost and new_p_cost <= current_p_cost):
        current_solution = new_solution
        current_cost = new_cost
        current_p_cost = new_p_cost
        current_d_cost = new_d_cost
        ic(step, current_cost)
    elif ((-(new_cost-current_cost)/temperature) > (np.log(rng.random()))) and (new_d_cost <= current_d_cost and new_p_cost <= current_p_cost):
        current_solution = new_solution
        current_cost = new_cost
        current_p_cost = new_p_cost
        current_d_cost = new_d_cost
        ic(step, current_cost)
    temperature*=0.99

if P(current_solution) != 0:
    print("Warning: final solution is invalid P =", P(current_solution))
if D(current_solution) != 0:
    print("Warning: final solution has duplicates D =", D(current_solution))
print("Final solution cost:", current_cost)

100%|██████████| 1000/1000 [02:36<00:00,  6.38it/s]

Final solution cost: -1691373.0
